<a href="https://colab.research.google.com/github/lee0721/basketball_yolov8/blob/main/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ 掛載 Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import os
import cv2
import torch
import glob
import subprocess
import numpy as np
from ultralytics import YOLO
from IPython.display import display, Image

# ✅ 設定環境變數（確保 UTF-8）
os.environ["LANG"] = "C.UTF-8"
os.environ["LC_ALL"] = "C.UTF-8"
os.environ["PYTHONIOENCODING"] = "UTF-8"
print("✅ 環境變數:", os.popen("echo $LANG").read().strip())  # **檢查是否正確設定**

# ✅ 安裝 ultralytics
subprocess.run(["pip", "install", "--upgrade", "ultralytics"], check=True)

# ✅ 設定 Google Drive 存放路徑
project_dir = "/content/gdrive/MyDrive/individual_project/proj/basketball_yolov8"
saved_models_dir = f"{project_dir}/saved_models"
video_source = f"{project_dir}/predicted_video.mp4"  # 你的影片來源
output_video = f"{project_dir}/predicted_output.mp4"  # 預測影片
output_frames_dir = f"{project_dir}/predicted_frames"  # 逐幀儲存圖片資料夾
os.makedirs(output_frames_dir, exist_ok=True)

# ✅ 讀取最新的 YOLOv8 模型
saved_models = sorted(glob.glob(f"{saved_models_dir}/yolov8_best_*.pt"), key=os.path.getmtime, reverse=True)
if not saved_models:
    print("⚠️ 沒有找到已儲存的模型，請確認是否有訓練並儲存模型！")
    exit()
latest_model_path = saved_models[0]
print(f"✅ 找到最新的已儲存模型: {latest_model_path}")

# ✅ 載入 YOLOv8 模型
model = YOLO(latest_model_path)

# ✅ 讀取影片
cap = cv2.VideoCapture(video_source)

# ✅ 設定影片輸出
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 設定 MP4 格式
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

frame_count = 0

# ✅ 逐幀處理影片
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # 如果影片結束，則停止

    # ✅ 使用 YOLOv8 進行預測
    results = model.predict(frame)

    # ✅ 在每一幀顯示 YOLO 預測結果
    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # 取得座標
            conf = box.conf[0].item()  # 取得信心分數
            label = r.names[int(box.cls[0])]  # 取得標籤

            # ✅ 繪製邊界框
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # ✅ 儲存處理後的影像
    frame_path = os.path.join(output_frames_dir, f"frame_{frame_count:04d}.jpg")
    cv2.imwrite(frame_path, frame)
    frame_count += 1

    # ✅ 寫入輸出影片
    out.write(frame)

# ✅ 釋放資源
cap.release()
out.release()
cv2.destroyAllWindows()
print(f"✅ 預測完成，影片已儲存至: {output_video}")

Streaming output truncated to the last 5000 lines.

0: 384x640 6 players, 154.9ms
Speed: 5.1ms preprocess, 154.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 players, 172.3ms
Speed: 4.7ms preprocess, 172.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 players, 146.8ms
Speed: 4.5ms preprocess, 146.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 players, 149.0ms
Speed: 8.3ms preprocess, 149.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 players, 144.0ms
Speed: 4.4ms preprocess, 144.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 players, 144.0ms
Speed: 4.6ms preprocess, 144.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 players, 159.7ms
Speed: 6.6ms preprocess, 159.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 players, 142.2ms
Speed: 7.0ms pr